In [1]:
import os
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import requests

script_path = os.getcwd()
result_path = os.path.join(script_path,'results')

In [2]:
dolos_site = "https://web.archive.org/web/20200808174908/https://www.professeur-alexandre-georges.info/dolos-list"
bealls_site = "https://beallslist.net/"
bealls_conf = "https://web.archive.org/web/20200515162250/https://libguides.caltech.edu/c.php?g=512665&p=3503029"

In [3]:
def get_domains(include_list):
    noheader = [x.replace("https://","http://").replace("http://","") for x in include_list]
    no_www = [x.replace("www.","").strip("/") for x in noheader]
    domains_only = []
    for eachlink in no_www:
        tmplink = eachlink.split("/")
        domain = tmplink[0]
        domains_only.append(domain)
    return domains_only

In [4]:
def get_dolos_list(dolos_site):
    r = requests.get(dolos_site)
    doloslist = []
    safelistkeys = ["professeur-alexandre","wikipedia","elsevier","dovepress","beallslist"]
    for link in BeautifulSoup(r.text, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            doloslist.append(link['href'])

    unarchiveit = [x.replace("https://web.archive.org/web/20200808174908/","") for x in doloslist]
    excludelist = []
    for eachkey in safelistkeys:
        tmplist = [x for x in unarchiveit if eachkey in x]
        excludelist.extend(tmplist)
    includelist = [x for x in unarchiveit if x not in excludelist]
    domains_only = get_domains(includelist)
    actionable_domains = ['@'+x for x in domains_only]
    return actionable_domains

In [5]:
def get_bealls_confs(bealls_conf):
    confs = requests.get(bealls_conf)
    first_conference = "https://web.archive.org/web/20200515162250/https://www.acavent.com/"
    beallslist = []
    for link in BeautifulSoup(confs.text, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            beallslist.append(link['href'])

    first_conf = beallslist.index(first_conference)
    conflist = beallslist[first_conf:]
    unarchiveit = [x.replace("https://web.archive.org/web/20200515162250/","") for x in conflist]
    confs_domains = get_domains(unarchiveit)
    actionable_confs = ['@'+x for x in confs_domains]
    return actionable_confs



In [6]:
def export_list(site_name,a_list):
    with open(os.path.join(result_path,f"{site_name}_domains.txt"),'w') as outwrite:
        for eachdomain in a_list:
            outwrite.write(eachdomain+"\n")

In [7]:
dolos_dirtylist = get_dolos_list(dolos_site)
dolos_domains = get_domains(dolos_dirtylist)
export_list("dolos_list",dolos_domains)

In [8]:
actionable_confs = get_bealls_confs(bealls_conf)
export_list("bealls_conferences",actionable_confs)
